In [35]:
import os
import pandas as pd
import re
import numpy as np
import glob
from stop_words import get_stop_words
from num2words import num2words 

In [36]:
path="/Users/simranpal20gmail.com/Documents/Apple/Data/RELOADS/quip"
os.chdir(path)
# quip_file = os.listdir(path)
# quip_file=quip_file[1]
quip_df = pd.read_excel('Q2_21_1.xlsx')

In [37]:
pay_week_ls=list(range(1,40))
fiscal_week_ls=list(range(14,53))
fiscal_week_ds = dict(zip(pay_week_ls, fiscal_week_ls))
def rearrage_weeks(weeks_impacted_ls,final_ls):
    for indx, pay_week in enumerate(weeks_impacted_ls):
        final_ls.append(str(fiscal_week_ds[int(pay_week)]))
    return final_ls

In [38]:
string_val_9='FY21W0'
string_val_above9='FY21W'
def week_number_9(no):
    if(int(no) <= 9):
        return (string_val_9 + no)
    else:
        return (string_val_above9 + no)

In [40]:
quip_df

,Date entered,Issue Type,Country,Apple HQ ID,Partner Name,Reporting Type,Raised by,Partner Contact,ST Capture Owner,Issues,Weeks impacted,Action required,Submission Type,Accepted in GDV (date),Status with STC,Status with TARS,Checked in Ent Dashboard,Comments,TARS Impact
37,26/03/2021,CQ Impact,CH,52671,Art Computer,End Cus Type & Name,Luca,NaN,Luc,W11 resubmission,[FY21W24],W11 file received and accepted,Resubmission,25/03/2021,Closed,Open,NaN,files replaced on 25/03,Ciara Murphy
39,2021-01-04 00:00:00,CQ Impact,AT,1630825,ACP HOLDING AT,End Cus Type & Name,Luca,NaN,Luca,Resubmission full quarter,"[FY21W14, FY21W15, FY21W16, FY21W17, FY21W18, ...",Files received and accepted,Resubmission,2021-01-04 00:00:00,Closed,Open,NaN,NaN,Emmet Healy
40,2021-01-04 00:00:00,CQ Impact,UK,387794,XMA,End Cus Type & Name,Melia,NaN,Melia,Duplication in previous data,[FY21W18],Files received and accepted,Resubmission,2021-01-04 00:00:00,Closed,Open,NaN,Files replaced 01/04/2021,Neil Kelly


In [39]:
quip_df=quip_df[(quip_df['Status with TARS'] == 'Open')].copy()
quip_df['Weeks impacted']=quip_df['Weeks impacted'].astype(str)
quip_df['Weeks impacted'] = quip_df.apply(lambda x: x['Weeks impacted'].split(","), axis=1)
quip_df['Weeks impacted'] = quip_df.apply(lambda x: rearrage_weeks(x['Weeks impacted'],final_ls=[]), axis=1)

for index,week_ls_row in quip_df.iterrows():
    quip_df.at[index,'Weeks impacted']= [week_number_9(x) for x in week_ls_row['Weeks impacted']]

In [30]:
path="/Users/simranpal20gmail.com/Documents/Apple/Data/RELOADS/GBI"
os.chdir(path)
GBI_df = pd.read_excel('RELOADs_Q2_21_1.xlsx')
# print(GBI_df.head())
GBI_df.columns=GBI_df.iloc[0]
GBI_df.drop(GBI_df.index[[0]],inplace=True)

In [31]:
print('GBI Dimension',GBI_df.shape)
# print('GBI Columns',GBI_df.columns)

GBI Dimension (892070, 27)


In [41]:
path="/Users/simranpal20gmail.com/Documents/Apple/Data/RELOADS"
os.chdir(path)
Admin_df = pd.read_excel('TARS Analysts - Q2FY21 Responsibilities & Partner Assignments.xlsx')
quip_admin_df = pd.merge(quip_df,Admin_df[['Partner TARS B2B Analyst','Apple HQ ID']],on='Apple HQ ID', how='left')

In [33]:
# GBI_df[(GBI_df['Apple HQ ID']=='2638065')]
# GBI_df[(GBI_df['Fiscal Week/Year (Name)']=='FY21W05') & (GBI_df['Apple HQ ID']==303923)]

In [42]:
path="/Users/simranpal20gmail.com/Documents/Apple/Data/RELOADS/Admins"
os.chdir(path)
for index, admin_row in quip_admin_df.iterrows():
    main_admin_df = pd.DataFrame(columns=GBI_df.columns)
    main_admin_df = GBI_df[(GBI_df['Fiscal Week/Year (Name)'].isin(admin_row['Weeks impacted'])) & (GBI_df['Apple HQ ID']==str(admin_row['Apple HQ ID']))]
    week_nums='-'.join(['W'+x[-2:] for x in admin_row['Weeks impacted']])
    excel_string=admin_row['Partner TARS B2B Analyst'].split(' ')[0]+' '+str(admin_row['Apple HQ ID'])+' '+week_nums+'.xlsx'
    w_write = pd.ExcelWriter(excel_string)
    main_admin_df.to_excel(w_write,index=False)
    w_write.save()